# Vector Databases

In the previous section of this workshop, we discussed the importance of embedding a language's semantics in vectors (typically with hundreds of dimensions). Modern AI applications such as RAG systems or search engines need to access vast amounts of vectors quickly.

In this part, we will delve into vector databases, a technology that permits indexing, storing, and accessing millions of vectors. 

## We need some dependencies

In [ ]:
import numpy as np
from scipy.spatial.distance import cityblock, euclidean, cosine
from qdrant_client import models, QdrantClient

import time

from movie_buddy.preprocessing.movies_dataset import get_movies_dataset
from sentence_transformers import SentenceTransformer

## What Does It Mean That Vectors Are Similar?

One of the main advantages of having text represented as a vector is that we can do some mathematics on it! Using different distance metrics, we can calculate how much two points in a multidimensional space are close to each other. The most commonly used in the context of NLP are: 

- Cosine Similarity
- Dot Product
- Euclidean Distance
- Manhattan Distance

## Are You a Math Geek? Let's see some formulas

If you are more interested in distance metrics mathematics, this paragraph is made for you. Otherwise, it is not necessary to understand the rest of the workshop; skip it!

### Distance Metrics Property
Do you want to invent new distance metrics? You can, but you must have some mathematical properties. So, given two vectors $a$ and $b$, a distance metric $d$ must be: 
1. **Non-negative**: $d(a, b) >= 0$;

2. **Symmetric**: $d(a, b) = d(a, b)$;

3. **Respect triangle inequality**: $d(a, b) <= d(a, r) + d(r, b)$ for all vectors $a$, $b$, $r$;

4. $d(p, q)=0$ only if $p=q$.

Okay, now let's see some distance formulas,: 


### Cosine Similarity
$$
d(a, b) = \cos(\theta) = \frac{\sum_{i=1}^{n} a_i b_i}{\sqrt{\sum_{i=1}^{n} a_i^2} \cdot \sqrt{\sum_{i=1}^{n} b_i^2}}
$$
### Dot Product
$$
d(a, b) = a \cdot b = \sum_{i=1}^{n} a_i b_i
$$

### Euclidean Distance
$$
d\left( a,b\right)   = \sqrt {\sum _{i=1}^{n}  \left( a_{i}-b_{i}\right)^2 } 
$$
### Manhattan Distance
$$
d(a, b) = \sum_{i=1}^{n} |a_i - b_i|
$$

**It is your turn!** In the following cell, try changing vector_a and vector_b to see how the different distances differ between the same vector!

In [ ]:
vector_a = np.array([0.9, 0.1, 0.23, 0.15])
vector_b = np.array([0.9, 0.30, 0.23, 0.25])

cosine_distance = cosine(vector_a, vector_b)
dot_distance = np.dot(vector_a, vector_b)
euclidean_dist = euclidean(vector_a, vector_b)
manhattan_dist = cityblock(vector_a, vector_b)

print(
    f"Cosine: {cosine_distance}\nDot: {dot_distance}\nEuclidean: {euclidean_dist}\nManhattan: {manhattan_dist}\n"
)

Try to guess in which case one metric is better than another. Which metric I would choose for which type of problem? 

## Why We Need Vector Databases? 

Can't we just choose a distance and get the k-closest vectors with a classical **K-Nearest-Neighbour** algorithm? 

Yes, we can! Let's try it. 

In [ ]:
def k_closest_vectors(target_vec, vectors, k):
    distances = np.sqrt(
        np.sum((vectors - target_vec) ** 2, axis=1)
    )  # Yes, you can implement euclidean distance by yourself.
    k_closest_indices = np.argsort(distances)[:k]
    closest_vectors = vectors[k_closest_indices]
    return closest_vectors

Let's forget for a moment of real sentences and we generate a random vector to search...

In [ ]:
VECTOR_DIM = 500
vector_to_search = np.random.uniform(low=-1, high=1, size=(VECTOR_DIM,))

... and a list of vector in which search

In [ ]:
N_VECTORS = 500000
K = 3

In [ ]:
vectors = np.random.uniform(low=0.0, high=1, size=(N_VECTORS, VECTOR_DIM))

In [ ]:
s_time = time.time()
closest_vectors = k_closest_vectors(vector_to_search, vectors, K)
e_time = time.time()
nn_total_time = e_time - s_time

f"To get the closer {K} vectors with naive K-Nearest-Neighbour algorithm took {nn_total_time:.2} sec"

Try increasing the N_VECTORS parameter. Did you see the problem? K-Nearest-Neighbour is a $O(N)$ algorithm that scales poorly with many vectors!

Instead, Vector Databases use efficient algorithms and data structures to store and search vectors. Of course, these algorithms are not some magic from a wizard. The general idea is to sacrifice some accuracy for a rapid vector search. This approach is called Approximate Nearest-Neighbour (ANN). In the year, researchers and engineers developed many ANN algorithms; the most popular nowadays are:

- HNSW: Hierarchical Navigable Small Worlds
    - Type: graph-based
    - [How it works?](https://towardsdatascience.com/similarity-search-part-4-hierarchical-navigable-small-world-hnsw-2aad4fe87d37)
    - original paper [here](https://arxiv.org/abs/1603.09320)
- ANNOY: Approximate Nearest Neighbour (Oh Yeah)
    - Type: tree-based
    - [How it works?](https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html)
    - Curious about code? repo [here](https://github.com/spotify/annoy)
- LSH: Locality Sensitive Hashing
    - Type: hash-based
    - [How it works?](https://www.notion.so/Vector-Database-101-1a1926611dc548c1a01a8f939a9dc42c?pvs=21)

We will use [Qdrant](https://qdrant.tech/) to build our RAG. Let's try to load our random vectors inside a qdrant collection.

In [ ]:
qdrant = QdrantClient(":memory:")

COLLECTION_NAME = "random_vectors"
DISTANCE = models.Distance.COSINE

qdrant.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(size=VECTOR_DIM, distance=DISTANCE),
)

In [ ]:
qdrant.upload_collection(collection_name=COLLECTION_NAME, vectors=vectors)

In [ ]:
s_time = time.time()
qdrant.search(collection_name=COLLECTION_NAME, query_vector=vector_to_search, limit=K)
e_time = time.time()
ann_total_time = e_time - s_time
f"To get the closer {K} vectors with a vector database took {ann_total_time:.2} sec"

## What About Movies?

You should now be and expert on vector database, qdrant and embeddings, try yourself to import our movies dataset overviews in qdrant:

In [ ]:
# TODO: REMOVE CODE THAT PARTECIPANTS SHOULD write

In [ ]:
movies_df = get_movies_dataset()
movies_df

Now we don't want to work with random data anymore let's instanciate our encoder...


In [ ]:
encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

.. and embed our movies overviews..

In [ ]:
# movies = encoder.encode(movies_df["overview"].tolist())

### Build the Collection

In [ ]:
qdrant = QdrantClient(":memory:")

COLLECTION_NAME = "movies"

qdrant.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), distance=models.Distance.COSINE
    ),
)

In [ ]:
records = [
    models.Record(
        id=idx,
        vector=encoder.encode(r["overview"]).tolist(),
        payload={
            "title": r["title"],
            "overview": r["overview"],
            "release_date": r["release_date"],
            "runtime": r["runtime"],
            "genre": r["genre"],
        },
    )
    for idx, r in movies_df.iterrows()
]

qdrant.upload_points(collection_name=COLLECTION_NAME, points=records)

Now, you could have fun searching your prefered movies!

In [ ]:
prompt = "Crazy Scientist save the planet from aliens"
encoded_prompt = encoder.encode(prompt)

In [ ]:
hits = qdrant.search(
    collection_name=COLLECTION_NAME, query_vector=encoded_prompt.tolist(), limit=10
)

In [ ]:
for res in hits:
    payload = res.payload
    print(
        "Title: {title}\nRelease date: {release_date}\nRuntime: {runtime}\n\n".format(
            title=payload["title"],
            release_date=payload["release_date"],
            runtime=payload["runtime"],
        )
    )